In [1]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import pymysql
import pandas as pd

In [2]:
conn = pymysql.connect(
    host="thirdtest.camwsondhmqr.us-east-2.rds.amazonaws.com",
    user="thirdtest",
    passwd="Qja1998+0325",
    db="ebdb"
)
cursor = conn.cursor()

In [21]:
cursor.execute('SELECT * FROM user WHERE world = 2');
rows = cursor.fetchall()
str(rows)[0:300]

"(('A1BQCRF5Q76YFY', 'Republicans', '5b7acb323ea88', '19,2,7,5,21,13,4,11,18,17,6,1,9,3,22,20,12,8,14,10,16,15', '', 2, 23), ('A125FN8IYVB93O', 'Democrats', '5b7ad2a8212d6', '18,16,17,14,21,3,6,1,11,5,19,15,8,9,22,13,10,7,20,2,4,12', '', 2, 23), ('A2KVYRNL2CK95F', 'Democrats', '5b7ad6afd81e8', '16,20"

In [22]:
df_users = pd.DataFrame( [[ij for ij in i] for i in rows] )
df_users.rename(
    columns={0:'Name', 1:'Political Stance', 2:'Session', 3:'Question Order', 4:'Current Question', 5:'World'},
    inplace=True
)
df_users = df_users.sort_values(['Name'], ascending=[1])
df_users

,Name,Political Stance,Session,Question Order,Current Question,World,6
1,A125FN8IYVB93O,Democrats,5b7ad2a8212d6,"18,16,17,14,21,3,6,1,11,5,19,15,8,9,22,13,10,7...",,2,23
0,A1BQCRF5Q76YFY,Republicans,5b7acb323ea88,"19,2,7,5,21,13,4,11,18,17,6,1,9,3,22,20,12,8,1...",,2,23
5,A1L5H5KX16J1LS,Democrats,5b7ae91899aea,"13,14,15,12,21,9,7,2,11,17,1,19,18,8,22,10,4,5...",,2,23
4,A2460MAJX0NA82,Democrats,5b7ae5725800f,"2,19,8,10,21,20,5,15,1,18,4,3,17,11,22,9,12,13...",,2,23
2,A2KVYRNL2CK95F,Democrats,5b7ad6afd81e8,"16,20,13,10,21,6,8,9,4,3,5,2,12,7,22,14,15,19,...",,2,23
3,A2TG7EEQ2XG2IK,Democrats,5b7ae05200182,"8,6,12,5,21,11,10,4,16,3,2,20,7,17,22,9,14,18,...",,2,23


In [23]:
cursor.execute('SELECT * FROM user_question_world_answer WHERE world_id != 1');
rows = cursor.fetchall()
str(rows)[0:300]

"(('A1BQCRF5Q76YFY', 2, 19, 'oppose'), ('A1BQCRF5Q76YFY', 2, 2, 'support'), ('A1BQCRF5Q76YFY', 2, 7, 'oppose'), ('A1BQCRF5Q76YFY', 2, 5, 'support'), ('A1BQCRF5Q76YFY', 2, 21, 'support'), ('A1BQCRF5Q76YFY', 2, 13, 'oppose'), ('A1BQCRF5Q76YFY', 2, 4, 'oppose'), ('A1BQCRF5Q76YFY', 2, 11, 'oppose'), ('A1"

In [24]:
df_responses = pd.DataFrame( [[ij for ij in i] for i in rows] )
df_responses.rename(
    columns={0:'Name', 1:'World', 2:'Question', 3:'Response'},
    inplace=True
)
df_responses = df_responses.sort_values(['Name'], ascending=[1])
df_responses = df_responses.pivot(index='Name', columns='Question', values='Response')
df_responses

Question,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,22
Name,,,,,,,,,,,,,,,,,,,,,
A125FN8IYVB93O,oppose,oppose,oppose,oppose,support,support,oppose,oppose,oppose,oppose,...,support,oppose,support,support,oppose,support,oppose,support,support,support
A1BQCRF5Q76YFY,oppose,support,oppose,oppose,support,oppose,oppose,oppose,support,oppose,...,oppose,support,support,support,oppose,oppose,oppose,support,support,support
A1L5H5KX16J1LS,oppose,oppose,support,support,support,oppose,support,support,support,oppose,...,oppose,oppose,support,support,oppose,oppose,oppose,support,oppose,support
A2460MAJX0NA82,oppose,support,support,support,support,support,oppose,oppose,support,support,...,support,oppose,support,oppose,support,oppose,oppose,support,oppose,oppose
A2KVYRNL2CK95F,oppose,oppose,oppose,oppose,oppose,support,oppose,oppose,oppose,support,...,oppose,oppose,support,oppose,support,oppose,oppose,support,support,oppose
A2TG7EEQ2XG2IK,oppose,support,support,support,support,support,oppose,support,oppose,support,...,oppose,oppose,oppose,oppose,oppose,oppose,oppose,support,support,oppose


In [116]:
df = pd.merge(
    left=df_users,
    right=df_responses,
    left_on='Name',
    right_on='Name'
)
df = df.replace('oppose', 0)
df = df.replace('support', 1)

finished = df['Current Question'].isin(list(range(1,23)))
finished = not any(list(finished))
if finished == True:
    df.drop(
        ['Current Question',
        'Session',
        '6_x'],
        axis=1)
    print(df.head())
else:
    print("Some participants did not complete the experiment:")
    print(df['Current Question'])

             Name Political Stance        Session  \
0  A125FN8IYVB93O        Democrats  5b7ad2a8212d6   
1  A1BQCRF5Q76YFY      Republicans  5b7acb323ea88   
2  A1L5H5KX16J1LS        Democrats  5b7ae91899aea   
3  A2460MAJX0NA82        Democrats  5b7ae5725800f   
4  A2KVYRNL2CK95F        Democrats  5b7ad6afd81e8   

                                      Question Order Current Question  World  \
0  18,16,17,14,21,3,6,1,11,5,19,15,8,9,22,13,10,7...                       2   
1  19,2,7,5,21,13,4,11,18,17,6,1,9,3,22,20,12,8,1...                       2   
2  13,14,15,12,21,9,7,2,11,17,1,19,18,8,22,10,4,5...                       2   
3  2,19,8,10,21,20,5,15,1,18,4,3,17,11,22,9,12,13...                       2   
4  16,20,13,10,21,6,8,9,4,3,5,2,12,7,22,14,15,19,...                       2   

   6_x  1  2  3 ...  13  14  15  16  17  18  19  20  21  22  
0   23  0  0  0 ...   1   0   1   1   0   1   0   1   1   1  
1   23  0  1  0 ...   0   1   1   1   0   0   0   1   1   1  
2   23  0  0  

In [43]:
df.to_csv('experiment_1_pilot.csv')